In [1]:
### RAG 

from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
from dotenv import load_dotenv

load_dotenv()
# embedding
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="harry_tokens",
    embedding_function=embed_model,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant=os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE"),
)

In [6]:
vector_store.get_by_ids(['045b1b78-ae31-4fa6-8cca-02665fc14515'])

[Document(id='045b1b78-ae31-4fa6-8cca-02665fc14515', metadata={'producer': 'calibre 3.27.1 [https://calibre-ebook.com]', 'subject': '', 'file_path': './harry.pdf', 'modDate': '', 'author': 'Rowling, J.K.', 'creationdate': '2019-02-21T16:04:23+00:00', 'moddate': '', 'creationDate': "D:20190221160423+00'00'", 'source': './harry.pdf', 'creator': 'calibre 3.27.1 [https://calibre-ebook.com]', 'total_pages': 3623, 'title': 'Harry Potter: The Complete Collection', 'format': 'PDF 1.4', 'page': 5, 'keywords': '', 'trapped': ''}, page_content='CONTENTS\nHarry Potter and the Sorcerer’s Stone\nHarry Potter and the Chamber of Secrets\nHarry Potter and the Prisoner of Azkaban\nHarry Potter and the Goblet of Fire\nHarry Potter and the Order of the Phoenix\nHarry Potter and the Half-Blood Prince\nHarry Potter and the Deathly Hallows')]

In [95]:
retriver = vector_store.as_retriever(search_kwargs = {"k": 20})

In [96]:
from langchain.tools import tool

@tool
def get_content(query:str):
    """Search and return information about Harry Potter Novel details."""
    details = retriver.invoke(query)
    return "\n\n".join([doc.page_content for doc in details])

In [97]:
get_content.invoke("who is Half-Blood Prince in Harry Potter?")

'my particular brand of reasoned argument is making much headway against\nGreyback’s insistence that we werewolves deserve blood, that we ought to\nrevenge ourselves on normal people.”\n“But you are normal!” said Harry fiercely. “You’ve just got a — a problem\n—”\nLupin burst out laughing. “Sometimes you remind me a lot of James. He\ncalled it my ‘furry little problem’ in company. Many people were under the\nimpression that I owned a badly behaved rabbit.”\nHe accepted a glass of eggnog from Mr. Weasley with a word of thanks,\nlooking slightly more cheerful. Harry, meanwhile, felt a rush of excitement:\nThis last mention of his father had reminded him that there was something he\nhad been looking forward to asking Lupin.\n“Have you ever heard of someone called the Half-Blood Prince?”\n“The Half-Blood what?”\n“Prince,” said Harry, watching him closely for signs of recognition.\n“There are no Wizarding princes,” said Lupin, now smiling. “Is this a title\nyou’re thinking of adopting? I sh

In [129]:
from langchain.chat_models import init_chat_model
from langgraph.graph import MessagesState
from langchain.messages import SystemMessage
llm = init_chat_model(
    model="gpt-5-nano"
)


def generate_query_or_respond(state:MessagesState):
       sys_msg = """Always use the get_content tool if the query is realted to Harry Potter novel or simply respond"""
       response = (llm
                   .bind_tools([get_content])
                   .invoke([SystemMessage(content=sys_msg)] + state["messages"]))
       
       print(response)
       
       return {"messages": [response]}
       


In [133]:
input = {"messages": [{"role": "user", "content": "who is Half-Blood Prince in Harry Potter?"}]}

out = generate_query_or_respond(input)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 603, 'prompt_tokens': 161, 'total_tokens': 764, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DDoUZehdkhkyDORUKs4YLcvqPWZIw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c9e70-eef1-7841-a421-f1080543c494-0' tool_calls=[{'name': 'get_content', 'args': {'query': 'Half-Blood Prince identity'}, 'id': 'call_cH0CXFT1rJzCrHpd7rVLlkp3', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 161, 'output_tokens': 603, 'total_tokens': 764, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 576}}


In [134]:
out    

{'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 603, 'prompt_tokens': 161, 'total_tokens': 764, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DDoUZehdkhkyDORUKs4YLcvqPWZIw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c9e70-eef1-7841-a421-f1080543c494-0', tool_calls=[{'name': 'get_content', 'args': {'query': 'Half-Blood Prince identity'}, 'id': 'call_cH0CXFT1rJzCrHpd7rVLlkp3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 161, 'output_tokens': 603, 'total_tokens': 764, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

In [135]:
for m in out["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_content (call_cH0CXFT1rJzCrHpd7rVLlkp3)
 Call ID: call_cH0CXFT1rJzCrHpd7rVLlkp3
  Args:
    query: Half-Blood Prince identity
